In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import mutual_info_regression

In [2]:
df_train_raw = pd.read_csv('train.csv')
df_test_raw = pd.read_csv('test.csv')
df_macro_data = pd.read_csv('macro.csv')
df_macro_euusd = df_macro_data[['timestamp', 'usdrub', 'eurrub']]

df_train_raw.shape, df_test_raw.shape

((30471, 292), (7662, 291))

In [3]:
def time_year(x):
    year = x.split('-')
    return int(year[0])

In [4]:
def time_month(x):
    month = x.split('-')
    return int(month[1])

In [5]:
def conv_to_bool(x):
    if x == 'yes':
        return 1
    else:
        return 0

In [6]:
df_train_raw = pd.merge(df_train_raw, df_macro_data, on='timestamp')
df_test_raw = pd.merge(df_test_raw, df_macro_data, on='timestamp')

In [7]:
s = 'oil_chemistry_raion'

te_bool = (df_test_raw[s].apply(conv_to_bool) == 1).sum()
tr_bool = (df_train_raw[s].apply(conv_to_bool) == 1).sum()

print(tr_bool, te_bool)

296 65


In [8]:
df_train_raw['ecology'].nunique()

5

In [8]:
na_train_only = []
for c in df_test_raw.columns:
    if pd.isnull(df_train_raw[c]).sum() != 0 and pd.isnull(df_test_raw[c]).sum() == 0:
        if type(df_train_raw[c][0]) != str:
            print(c, pd.isnull(df_train_raw[c]).sum())
            na_train_only.append(c)
print(na_train_only)

floor 167
max_floor 9572
material 9572
num_room 9572
kitch_sq 9572
['floor', 'max_floor', 'material', 'num_room', 'kitch_sq']


In [17]:
(pd.isnull(df_train_raw['floor']) & pd.isnull(df_train_raw['max_floor'])).sum()

167

In [18]:
# na_train_only = []
for c in df_test_raw.columns:
    if pd.isnull(df_train_raw[c]).sum() == 0 and pd.isnull(df_test_raw[c]).sum() != 0:
        if type(df_train_raw[c][0]) != str:
            print(c, pd.isnull(df_test_raw[c]).sum())
#             na_train_only.append(c)
# print(na_train_only)

green_part_2000 19
income_per_cap 3983
salary 3983
salary_growth 3983
retail_trade_turnover 3983
retail_trade_turnover_per_cap 3983
retail_trade_turnover_growth 3983
labor_force 3983
unemployment 3983
employment 3983
invest_fixed_capital_per_cap 3983
invest_fixed_assets 3983
pop_natural_increase 3983
childbirth 3983
mortality 3983
average_life_exp 3983
load_of_teachers_school_per_teacher 3983
students_state_oneshift 3983
provision_nurse 3983
load_on_doctors 3983
turnover_catering_per_cap 3983
seats_theather_rfmin_per_100000_cap 3983
bandwidth_sports 3983
apartment_fund_sqm 3983


In [9]:
numeric_cols = []
for c in df_test_raw.columns:
    if pd.isnull(df_train_raw[c]).sum() == 0 and pd.isnull(df_test_raw[c]).sum() == 0:
        if type(df_train_raw[c][0]) != str:
            print("'" + c + "',")
            numeric_cols.append(c)

'id',
'full_sq',
'area_m',
'raion_popul',
'green_zone_part',
'indust_part',
'children_preschool',
'preschool_education_centers_raion',
'children_school',
'school_education_centers_raion',
'school_education_centers_top_20_raion',
'healthcare_centers_raion',
'university_top_20_raion',
'sport_objects_raion',
'additional_education_raion',
'culture_objects_top_25_raion',
'shopping_centers_raion',
'office_raion',
'full_all',
'male_f',
'female_f',
'young_all',
'young_male',
'young_female',
'work_all',
'work_male',
'work_female',
'ekder_all',
'ekder_male',
'ekder_female',
'0_6_all',
'0_6_male',
'0_6_female',
'7_14_all',
'7_14_male',
'7_14_female',
'0_17_all',
'0_17_male',
'0_17_female',
'16_29_all',
'16_29_male',
'16_29_female',
'0_13_all',
'0_13_male',
'0_13_female',
'ID_metro',
'metro_min_avto',
'metro_km_avto',
'kindergarten_km',
'school_km',
'park_km',
'green_zone_km',
'industrial_km',
'water_treatment_km',
'cemetery_km',
'incineration_km',
'railroad_station_avto_km',
'railroad_station_avt

In [14]:
corr_c = []
sp_corr_c = []

for c in numeric_cols:
    if c != 'id':
        cc = df_train_raw[c].corr(df_train_raw['price_doc'] / df_train_raw['eurrub'])
        corr_c.append([c, cc])
        cc = df_train_raw[c].corr(df_train_raw['price_doc'] / df_train_raw['eurrub'], method='spearman')
        sp_corr_c.append([c, cc])

In [15]:
corr_c = sorted(corr_c, key=lambda x: -abs(x[1]))
sp_corr_c = sorted(sp_corr_c, key=lambda x: -abs(x[1]))

In [16]:
corr_c[:170]

[['full_sq', 0.33430970610986782],
 ['sport_count_5000', 0.28798194993128368],
 ['sport_count_3000', 0.28705472351720512],
 ['trc_count_5000', 0.286517726768859],
 ['zd_vokzaly_avto_km', -0.28275042220905011],
 ['sadovoe_km', -0.28046967833282999],
 ['sport_count_2000', 0.27717117280374737],
 ['bulvar_ring_km', -0.27612741635119731],
 ['kremlin_km', -0.27556537057610297],
 ['ttk_km', -0.27058183387321977],
 ['trc_sqm_5000', 0.26150518180787069],
 ['nuclear_reactor_km', -0.25951782186054223],
 ['sport_count_1500', 0.25692451572698266],
 ['office_sqm_5000', 0.25622059687851173],
 ['sport_objects_raion', 0.24665530223879836],
 ['trc_count_3000', 0.24108211604063981],
 ['stadium_km', -0.23834441322144198],
 ['cafe_count_5000_price_1000', 0.2310263136088922],
 ['detention_facility_km', -0.22951796825507947],
 ['basketball_km', -0.22871309715441912],
 ['cafe_count_5000_price_1500', 0.22308524363504317],
 ['office_km', -0.22292887890255433],
 ['cafe_count_5000', 0.22168543087709422],
 ['cafe_

In [17]:
correl_15 = []
sp_correl_15 = []

for c in corr_c:
    if abs(c[1]) > 0.15:
        correl_15.append(c[0])

for c in sp_corr_c:
    if abs(c[1]) > 0.25:
        sp_correl_15.append(c[0])

In [18]:
print(correl_15)

['full_sq', 'sport_count_5000', 'sport_count_3000', 'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'sport_count_2000', 'bulvar_ring_km', 'kremlin_km', 'ttk_km', 'trc_sqm_5000', 'nuclear_reactor_km', 'sport_count_1500', 'office_sqm_5000', 'sport_objects_raion', 'trc_count_3000', 'stadium_km', 'cafe_count_5000_price_1000', 'detention_facility_km', 'basketball_km', 'cafe_count_5000_price_1500', 'office_km', 'cafe_count_5000', 'cafe_count_5000_na_price', 'university_km', 'trc_sqm_3000', 'cafe_count_5000_price_500', 'workplaces_km', 'cafe_count_5000_price_2500', 'office_sqm_3000', 'theater_km', 'swim_pool_km', 'thermal_power_plant_km', 'office_count_5000', 'catering_km', 'exhibition_km', 'church_count_5000', 'office_sqm_2000', 'cafe_count_5000_price_high', 'radiation_km', 'cafe_count_5000_price_4000', 'big_church_km', 'school_education_centers_raion', 'sport_count_1000', 'fitness_km', 'metro_min_avto', 'market_count_5000', 'museum_km', 'park_km', 'big_church_count_5000', 'leisure_cou

In [19]:
print(sp_correl_15)

['full_sq', 'cafe_count_2000', 'cafe_count_3000_price_2500', 'cafe_count_3000_price_1500', 'cafe_count_3000', 'cafe_count_2000_price_1000', 'cafe_count_3000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_2000_price_1500', 'cafe_count_5000', 'cafe_count_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_1000', 'trc_count_5000', 'trc_count_3000', 'cafe_count_2000_price_2500', 'sport_count_3000', 'sport_count_5000', 'cafe_count_5000_price_500', 'cafe_count_1500_price_1000', 'cafe_count_3000_price_500', 'sport_count_2000', 'cafe_count_1000', 'cafe_count_5000_na_price', 'cafe_count_2000_price_500', 'cafe_count_1500_price_1500', 'ttk_km', 'sadovoe_km', 'office_count_5000', 'kremlin_km', 'zd_vokzaly_avto_km', 'office_sqm_5000', 'trc_count_2000', 'trc_sqm_3000', 'bulvar_ring_km', 'trc_sqm_5000', 'sport_count_1500', 'nuclear_reactor_km', 'office_count_3000', 'sport_objects_raion', 'metro_km_avto', 'cafe_count_1500_price_500', 'cafe_count_1500_price_2500', 'workplaces_km', 'big_

In [20]:
both_corr = []
for k in correl_15:
    if k in sp_correl_15:
        both_corr.append(k)
print(both_corr)

['full_sq', 'sport_count_5000', 'sport_count_3000', 'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'sport_count_2000', 'bulvar_ring_km', 'kremlin_km', 'ttk_km', 'trc_sqm_5000', 'nuclear_reactor_km', 'sport_count_1500', 'office_sqm_5000', 'sport_objects_raion', 'trc_count_3000', 'stadium_km', 'cafe_count_5000_price_1000', 'detention_facility_km', 'basketball_km', 'cafe_count_5000_price_1500', 'office_km', 'cafe_count_5000', 'cafe_count_5000_na_price', 'university_km', 'trc_sqm_3000', 'cafe_count_5000_price_500', 'workplaces_km', 'cafe_count_5000_price_2500', 'office_sqm_3000', 'swim_pool_km', 'thermal_power_plant_km', 'office_count_5000', 'catering_km', 'exhibition_km', 'church_count_5000', 'office_sqm_2000', 'cafe_count_5000_price_high', 'cafe_count_5000_price_4000', 'big_church_km', 'school_education_centers_raion', 'sport_count_1000', 'fitness_km', 'metro_min_avto', 'market_count_5000', 'park_km', 'big_church_count_5000', 'leisure_count_5000', 'office_sqm_1500', 'ekder_male', 

In [29]:
corr15_only = []
for k in corr_c:
    if k[0] not in both_corr and abs(k[1]) > 0.2:
        corr15_only.append(k)
corr15_only

[['theater_km', -0.21276866009041509]]

In [27]:
sp_corr15_only = []
for k in sp_corr_c:
    if k[0] not in both_corr and k[1] > 0.3:
        sp_corr15_only.append(k)
sp_corr15_only

[['cafe_count_2000', 0.36325400039557892],
 ['cafe_count_3000_price_2500', 0.35521400027549871],
 ['cafe_count_3000_price_1500', 0.35488503845598451],
 ['cafe_count_3000', 0.35370671909241375],
 ['cafe_count_2000_price_1000', 0.35238170130835417],
 ['cafe_count_2000_price_1500', 0.34623034431255423],
 ['cafe_count_1500', 0.3456888606061973],
 ['cafe_count_2000_price_2500', 0.33563864691322887],
 ['cafe_count_1500_price_1000', 0.33082136911641363],
 ['cafe_count_3000_price_500', 0.3281547617692615],
 ['cafe_count_1000', 0.32635631013245353],
 ['cafe_count_2000_price_500', 0.32430519782681055],
 ['cafe_count_1500_price_1500', 0.322575384752528],
 ['office_count_3000', 0.30914784275262486],
 ['cafe_count_1500_price_500', 0.30475376025509898],
 ['cafe_count_1500_price_2500', 0.30446303076649783]]

In [40]:
len(both_corr), len(correl_15), len(corr15_only)

(36, 47, 11)

In [27]:
sp_corr_c[:80]

[['full_sq', 0.48729718654049942],
 ['cafe_count_2000', 0.36325400039557892],
 ['cafe_count_3000_price_2500', 0.35521400027549871],
 ['cafe_count_3000_price_1500', 0.35488503845598451],
 ['cafe_count_3000', 0.35370671909241375],
 ['cafe_count_2000_price_1000', 0.35238170130835417],
 ['cafe_count_3000_price_1000', 0.34960677988105399],
 ['cafe_count_5000_price_1500', 0.34645164919035398],
 ['cafe_count_2000_price_1500', 0.34623034431255423],
 ['cafe_count_5000', 0.34581665212530993],
 ['cafe_count_1500', 0.3456888606061973],
 ['cafe_count_5000_price_2500', 0.34504048162427503],
 ['cafe_count_5000_price_1000', 0.33905231304010414],
 ['trc_count_5000', 0.33650959572825734],
 ['trc_count_3000', 0.33644011987488376],
 ['cafe_count_2000_price_2500', 0.33563864691322887],
 ['sport_count_3000', 0.33529439803424077],
 ['sport_count_5000', 0.33412202649342493],
 ['cafe_count_5000_price_500', 0.33107028497683127],
 ['cafe_count_1500_price_1000', 0.33082136911641363],
 ['cafe_count_3000_price_500'